In [1]:
import pandas as pd
import numpy as np

In [2]:
import pandas as pd
import numpy as np
import ast
import json


In [3]:

def clean_state_action_csv(input_file, output_file, config_file):
    # Load the config.json file
    with open(config_file, 'r') as f:
        config = json.load(f)
    
    # Extract values of M, L, and N from the config
    M = len(config['session']['codes']) + 1  # Number of assets + 1 for cash or risk-free asset
    L = int(config['session']['agents'][2])  # Look-back window length
    N = len(config['session']['features'])  # Number of features per asset
    
    # Load the CSV file
    df = pd.read_csv(input_file)
    
    # Function to parse the string representation of arrays
    def parse_array(array_str):
        return np.array(ast.literal_eval(array_str.replace('\n', '').replace('  ', ',').replace(' ', ',')))
    
    # print(df.head())
    
    # Parse the State and Action columns
    df['State'] = df['State'].apply(parse_array)
    df['Action'] = df['Action'].apply(parse_array)

    # print(df.head())
    
   # Flatten the State and Action columns
    flattened_data = []
    for _, row in df.iterrows():
        action = row['Action'].flatten()
        state = row['State'].flatten()
        flattened_row = np.concatenate((action, state))
        flattened_data.append(flattened_row)
    
    # print(flattened_data)
    
    # Create column names
    # Action columns with asset names
    action_columns = []
    for i in range(M):
        asset_name = config['session']['codes'][i-1] if i > 0 else 'Cash'
        column_name = f'Action_{asset_name}'
        action_columns.append(column_name)
    state_columns = []
    for i in range(M):
        for j in range(L):
            for k in range(N):
                # Add the feature name and asset name to the column
                feature_name = config['session']['features'][k]
                asset_name = config['session']['codes'][i-1] if i > 0 else 'Cash'
                column_name = f'State_{asset_name}_{feature_name}_L{j+1}'
                state_columns.append(column_name)
    columns = action_columns + state_columns
    print(columns)
    # Create a new DataFrame with the flattened data
    cleaned_df = pd.DataFrame(flattened_data, columns=columns)
    
    # Save the cleaned DataFrame to a new CSV file
    cleaned_df.to_csv(output_file, index=False)


In [4]:
def merge_state_action_results(cleaned_state_action_file, results_file, output_file,config_file):
    # Load the config.json file
    with open(config_file, 'r') as f:
        config = json.load(f)
    
    # Extract values of M, L, and N from the config
    M = len(config['session']['codes']) + 1  # Number of assets + 1 for cash or risk-free asset
    
    # Load the cleaned state-action CSV
    state_action_df = pd.read_csv(cleaned_state_action_file)
    
    # Load the results CSV
    results_df = pd.read_csv(results_file)
    
    # Drop the 'weight' column from the results CSV if it exists
    if 'Weight' in results_df.columns:
        results_df = results_df.drop(columns=['Weight'])
    
    price_columns = []
    # Divide the price column in M columns by the comma
    for i in range(M):
        # Add the asset name to the column from the config
        asset_name = config['session']['codes'][i-1] if i > 0 else 'Cash'
        price_columns.append(f'Price_{asset_name}')
    results_df[price_columns] = results_df['Price'].str.split(',', expand=True)
    results_df.drop('Price', axis=1, inplace=True)

    # Drop the first column of the results_df
    results_df.drop(results_df.columns[0], axis=1, inplace=True)

    # Merge the two dataframes on their indices (assuming they align correctly)
    merged_df = pd.concat([state_action_df, results_df], axis=1)
    
    # Save the merged dataframe to a new CSV file
    merged_df.to_csv(output_file, index=False)



In [5]:


# Example usage
input_file = '../state_action_recopilation.csv'
output_file = '../explainability_data/cleaned_state_action.csv'
config_file = '../config.json'

clean_state_action_csv(input_file, output_file, config_file)


['Action_Cash', 'Action_AAPL', 'Action_ADBE', 'Action_BABA', 'Action_SNE', 'Action_V', 'State_Cash_close_L1', 'State_Cash_high_L1', 'State_Cash_low_L1', 'State_Cash_open_L1', 'State_AAPL_close_L1', 'State_AAPL_high_L1', 'State_AAPL_low_L1', 'State_AAPL_open_L1', 'State_ADBE_close_L1', 'State_ADBE_high_L1', 'State_ADBE_low_L1', 'State_ADBE_open_L1', 'State_BABA_close_L1', 'State_BABA_high_L1', 'State_BABA_low_L1', 'State_BABA_open_L1', 'State_SNE_close_L1', 'State_SNE_high_L1', 'State_SNE_low_L1', 'State_SNE_open_L1', 'State_V_close_L1', 'State_V_high_L1', 'State_V_low_L1', 'State_V_open_L1']


In [6]:

# Example usage
cleaned_state_action_file = '../explainability_data/cleaned_state_action.csv'
results_file = '../result1-49.73244555831117.csv'
output_file = '../explainability_data/state_action_results.csv'
config_file = '../config.json'

merge_state_action_results(cleaned_state_action_file, results_file, output_file,config_file)


In [7]:
data = pd.read_csv('../explainability_data/state_action_recopilation.csv',sep=',')

data.head()

,State,Action
0,[[[[1.0 1.0 1.0 1.0 ]] [[0.96606804 0.9683129 ...,[[0.25030902 0.5672448 0.05410691 0.03054755 0...
1,[[[[1.0 1.0 1.0 1.0 ]] [[0.96710413 0.9778104 ...,[[0.07127292 0.03257969 0.45481876 0.16591062 ...
2,[[[[1.0 1.0 1.0 1.0 ]] [[0.94327405 0.97245726...,[[0.1278808 0.26960662 0.0436136 0.21800773 0....
3,[[[[1.0 1.0 1.0 1.0 ]] [[0.95164911 0.97392506...,[[0.4222946 0.20674424 0.06997138 0.09220499 0...
4,[[[[1.0 1.0 1.0 1.0 ]] [[0.94802279 0.95398031...,[[0.14464499 0.06648933 0.4907311 0.20334779 0...


In [8]:
# From data column State eliminate all \n
data['State'] = data['State'].str.replace('\n','')

data.head()

,State,Action
0,[[[[1.0 1.0 1.0 1.0 ]] [[0.96606804 0.9683129 ...,[[0.25030902 0.5672448 0.05410691 0.03054755 0...
1,[[[[1.0 1.0 1.0 1.0 ]] [[0.96710413 0.9778104 ...,[[0.07127292 0.03257969 0.45481876 0.16591062 ...
2,[[[[1.0 1.0 1.0 1.0 ]] [[0.94327405 0.97245726...,[[0.1278808 0.26960662 0.0436136 0.21800773 0....
3,[[[[1.0 1.0 1.0 1.0 ]] [[0.95164911 0.97392506...,[[0.4222946 0.20674424 0.06997138 0.09220499 0...
4,[[[[1.0 1.0 1.0 1.0 ]] [[0.94802279 0.95398031...,[[0.14464499 0.06648933 0.4907311 0.20334779 0...


In [13]:
# Clean the State column
data['State'] = data['State'].str.replace('[[[[','[')
data['State'] = data['State'].str.replace(']]]]',']')

data['Action'] = data['Action'].str.replace('[[','')
data['Action'] = data['Action'].str.replace(']]','')

data['State'] = data['State'].str.replace('[[','[')
data['State'] = data['State'].str.replace(']]',']')

data['State'] = data['State'].str.replace(']   [',' ')

data['State'] = data['State'].str.replace(']  [',' ],[')

data.head()

                                                 State  \
0    [1.0 1.0 1.0 1.0 ] [0.96606804 0.9683129 0.938...   
1    [1.0 1.0 1.0 1.0 ] [0.96710413 0.9778104 0.951...   
2    [1.0 1.0 1.0 1.0 ] [0.94327405 0.97245726 0.93...   
3    [1.0 1.0 1.0 1.0 ] [0.95164911 0.97392506 0.94...   
4    [1.0 1.0 1.0 1.0 ] [0.94802279 0.95398031 0.93...   
..                                                 ...   
493  [1.0 1.0 1.0 1.0 ] [1.00975652 1.01450527 1.00...   
494  [1.0 1.0 1.0 1.0 ] [1.01070627 1.01364186 1.00...   
495  [1.0 1.0 1.0 1.0 ] [1.00405802 1.00595752 0.99...   
496  [1.0 1.0 1.0 1.0 ] [1.00604386 1.00604386 0.99...   
497  [1.0 1.0 1.0 1.0 ] [1.01243309 1.01709549 1.00...   

                                                Action  
0    0.25030902 0.5672448 0.05410691 0.03054755 0.0...  
1    0.07127292 0.03257969 0.45481876 0.16591062 0....  
2    0.1278808 0.26960662 0.0436136 0.21800773 0.12...  
3    0.4222946 0.20674424 0.06997138 0.09220499 0.0...  
4    0.14464499 0.

In [16]:
# Divide the state column in 6 columns by the comma
data[['State1','State2','State3','State4','State5','State6']] = data['State'].str.split(',',expand=True)
data.drop('State',axis=1,inplace=True)
data.head()

KeyError: 'State'

In [42]:
data['Action'] = data['Action'].str.replace(' ',',')
data['Action'] = data['Action'].str.replace(',,,',',')
data['Action'] = data['Action'].str.replace(',,',',')

# Delete the last comma
data['Action'] = data['Action'].str[:-1]

In [43]:

data['Action'][1]


'0.06870089,0.10743111,0.60890466,0.1358273,0.02387627,0.0552597'

In [44]:
data.head()

,Action,State1,State2,State3,State4,State5,State6
0,"0.16156216,0.1718296,0.277353,0.15543011,0.106...",[1. 1. 1. ],[0.94327405 0.95164911 0.94802279 ],[0.69082079 0.68567266 0.67984458 ],[1.15727138 1.14759139 1.1340394 ],[0.76204067 0.75561898 0.73849447 ],[0.83311971 0.83555499 0.81889259]
1,"0.06870089,0.10743111,0.60890466,0.1358273,0.0...",[1. 1. 1. ],[0.95164911 0.94802279 0.92229321 ],[0.68567266 0.67984458 0.68275862 ],[1.14759139 1.1340394 1.09679991 ],[0.75561898 0.73849447 0.74669996 ],[0.83555499 0.81889259 0.81107408]
2,"0.30272743,0.22010283,0.11606488,0.14536794,0....",[1. 1. 1. ],[0.94802279 0.92229321 0.91512692 ],[0.67984458 0.68275862 0.69674599 ],[1.1340394 1.09679991 1.10340508 ],[0.73849447 0.74669996 0.73742419 ],[0.81889259 0.81107408 0.81697001]
3,"0.3973439,0.06948966,0.15375659,0.04240716,0.1...",[1. 1. 1. ],[0.92229321 0.91512692 0.93869798 ],[0.68275862 0.69674599 0.69441476 ],[1.09679991 1.10340508 1.13927799 ],[0.74669996 0.73742419 0.7427756 ],[0.81107408 0.81697001 0.8229941 ]
4,"0.14738251,0.21148583,0.06130568,0.09160802,0....",[1. 1. 1. ],[0.91512692 0.93869798 0.94586427 ],[0.69674599 0.69441476 0.69490044 ],[1.10340508 1.13927799 1.17628972 ],[0.73742419 0.7427756 0.7816625 ],[0.81697001 0.8229941 0.824404 ]


In [45]:

# Divide the action column in 6 columns by the space
data[['Action1','Action2','Action3','Action4','Action5','Action6']] = data['Action'].str.split(',',expand=True)
print(data['Action'].str.split(',',expand=True).shape[1])

data.drop('Action',axis=1,inplace=True)
# data.drop('Action7',axis=1,inplace=True)
data.head()

6


,State1,State2,State3,State4,State5,State6,Action1,Action2,Action3,Action4,Action5,Action6
0,[1. 1. 1. ],[0.94327405 0.95164911 0.94802279 ],[0.69082079 0.68567266 0.67984458 ],[1.15727138 1.14759139 1.1340394 ],[0.76204067 0.75561898 0.73849447 ],[0.83311971 0.83555499 0.81889259],0.16156216,0.1718296,0.277353,0.15543011,0.10646321,0.1273619
1,[1. 1. 1. ],[0.95164911 0.94802279 0.92229321 ],[0.68567266 0.67984458 0.68275862 ],[1.14759139 1.1340394 1.09679991 ],[0.75561898 0.73849447 0.74669996 ],[0.83555499 0.81889259 0.81107408],0.06870089,0.10743111,0.60890466,0.1358273,0.02387627,0.0552597
2,[1. 1. 1. ],[0.94802279 0.92229321 0.91512692 ],[0.67984458 0.68275862 0.69674599 ],[1.1340394 1.09679991 1.10340508 ],[0.73849447 0.74669996 0.73742419 ],[0.81889259 0.81107408 0.81697001],0.30272743,0.22010283,0.11606488,0.14536794,0.03913353,0.1766034
3,[1. 1. 1. ],[0.92229321 0.91512692 0.93869798 ],[0.68275862 0.69674599 0.69441476 ],[1.09679991 1.10340508 1.13927799 ],[0.74669996 0.73742419 0.7427756 ],[0.81107408 0.81697001 0.8229941 ],0.3973439,0.06948966,0.15375659,0.04240716,0.11848463,0.2185181
4,[1. 1. 1. ],[0.91512692 0.93869798 0.94586427 ],[0.69674599 0.69441476 0.69490044 ],[1.10340508 1.13927799 1.17628972 ],[0.73742419 0.7427756 0.7816625 ],[0.81697001 0.8229941 0.824404 ],0.14738251,0.21148583,0.06130568,0.09160802,0.29695398,0.1912638


In [46]:
# Clean each state column
for i in range(1,7):
    data['State'+str(i)] = data['State'+str(i)].str.replace('[','')
    data['State'+str(i)] = data['State'+str(i)].str.replace(']','')
    data['State'+str(i)] = data['State'+str(i)].str.replace('   ',' ')
    data['State'+str(i)] = data['State'+str(i)].str.replace('  ',' ')
    data['State'+str(i)] = data['State'+str(i)].str.replace('  ',' ')
    data['State'+str(i)] = data['State'+str(i)].str.replace(' ',',')
    data['State'+str(i)] = data['State'+str(i)].str.replace('\n','')
    # delete the last comma
    data['State'+str(i)] = data['State'+str(i)].str[:-1]
    data[['State'+str(i)+'.1','State'+str(i)+'.2','State'+str(i)+'.3']] = data['State'+str(i)].str.split(',',expand=True)
    data.drop('State'+str(i),axis=1,inplace=True)

data

,Action1,Action2,Action3,Action4,Action5,Action6,State1.1,State1.2,State1.3,State2.1,...,State3.3,State4.1,State4.2,State4.3,State5.1,State5.2,State5.3,State6.1,State6.2,State6.3
0,0.16156216,0.1718296,0.277353,0.15543011,0.10646321,0.1273619,1.,1.,1.,0.94327405,...,0.67984458,1.15727138,1.14759139,1.1340394,0.76204067,0.75561898,0.73849447,0.83311971,0.83555499,0.8188925
1,0.06870089,0.10743111,0.60890466,0.1358273,0.02387627,0.0552597,1.,1.,1.,0.95164911,...,0.68275862,1.14759139,1.1340394,1.09679991,0.75561898,0.73849447,0.74669996,0.83555499,0.81889259,0.8110740
2,0.30272743,0.22010283,0.11606488,0.14536794,0.03913353,0.1766034,1.,1.,1.,0.94802279,...,0.69674599,1.1340394,1.09679991,1.10340508,0.73849447,0.74669996,0.73742419,0.81889259,0.81107408,0.8169700
3,0.3973439,0.06948966,0.15375659,0.04240716,0.11848463,0.2185181,1.,1.,1.,0.92229321,...,0.69441476,1.09679991,1.10340508,1.13927799,0.74669996,0.73742419,0.7427756,0.81107408,0.81697001,0.8229941
4,0.14738251,0.21148583,0.06130568,0.09160802,0.29695398,0.1912638,1.,1.,1.,0.91512692,...,0.69490044,1.10340508,1.13927799,1.17628972,0.73742419,0.7427756,0.7816625,0.81697001,0.8229941,0.824404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,0.0978908,0.27915582,0.16938849,0.04015029,0.09862785,0.3147867,1.,1.,1.,1.00129511,...,1.02739194,1.00979387,1.01275481,1.0231181,1.01462719,1.02889761,1.02533,1.00422968,1.00102538,1.0043578
490,0.02119086,0.13486058,0.09466165,0.06963388,0.37395534,0.3056976,1.,1.,1.,1.00707995,...,1.02486644,1.01275481,1.0231181,1.01639904,1.02889761,1.02533,1.01641099,1.00102538,1.00435786,1.0015380
491,0.18371621,0.18056609,0.12542987,0.09980048,0.22307195,0.1874154,1.,1.,1.,1.00975652,...,1.01719281,1.0231181,1.01639904,0.98849789,1.02533,1.01641099,1.01141634,1.00435786,1.00153807,0.9984619
492,0.12759693,0.2779875,0.17668611,0.1506408,0.1002835,0.1668051,1.,1.,1.,1.01070627,...,1.02010685,1.01639904,0.98849789,0.98838401,1.01641099,1.01141634,1.01534071,1.00153807,0.99846193,1.0016662


In [47]:
data.dropna(inplace=True)

In [48]:
# Find and drop any rows with value '' in the columns
data[data.isin(['']).any(axis=1)]
data.drop(data[data.isin(['']).any(axis=1)].index,inplace=True)


In [49]:

# Convert all columns in data to float
data = data.astype(float)


In [50]:
data.head()

,Action1,Action2,Action3,Action4,Action5,Action6,State1.1,State1.2,State1.3,State2.1,...,State3.3,State4.1,State4.2,State4.3,State5.1,State5.2,State5.3,State6.1,State6.2,State6.3
0,0.161562,0.171830,0.277353,0.155430,0.106463,0.127362,1.0,1.0,1.0,0.943274,...,0.679845,1.157271,1.147591,1.134039,0.762041,0.755619,0.738494,0.833120,0.835555,0.818893
1,0.068701,0.107431,0.608905,0.135827,0.023876,0.055260,1.0,1.0,1.0,0.951649,...,0.682759,1.147591,1.134039,1.096800,0.755619,0.738494,0.746700,0.835555,0.818893,0.811074
2,0.302727,0.220103,0.116065,0.145368,0.039134,0.176603,1.0,1.0,1.0,0.948023,...,0.696746,1.134039,1.096800,1.103405,0.738494,0.746700,0.737424,0.818893,0.811074,0.816970
3,0.397344,0.069490,0.153757,0.042407,0.118485,0.218518,1.0,1.0,1.0,0.922293,...,0.694415,1.096800,1.103405,1.139278,0.746700,0.737424,0.742776,0.811074,0.816970,0.822994
4,0.147383,0.211486,0.061306,0.091608,0.296954,0.191264,1.0,1.0,1.0,0.915127,...,0.694900,1.103405,1.139278,1.176290,0.737424,0.742776,0.781663,0.816970,0.822994,0.824404


In [33]:
# # Create a column action that is an numeric array of the actions
# data['Action'] = data[['Action1','Action2','Action3','Action4','Action5','Action6']].values.tolist()
# data.drop(['Action1','Action2','Action3','Action4','Action5','Action6'],axis=1,inplace=True)

In [51]:
data.head()

,Action1,Action2,Action3,Action4,Action5,Action6,State1.1,State1.2,State1.3,State2.1,...,State3.3,State4.1,State4.2,State4.3,State5.1,State5.2,State5.3,State6.1,State6.2,State6.3
0,0.161562,0.171830,0.277353,0.155430,0.106463,0.127362,1.0,1.0,1.0,0.943274,...,0.679845,1.157271,1.147591,1.134039,0.762041,0.755619,0.738494,0.833120,0.835555,0.818893
1,0.068701,0.107431,0.608905,0.135827,0.023876,0.055260,1.0,1.0,1.0,0.951649,...,0.682759,1.147591,1.134039,1.096800,0.755619,0.738494,0.746700,0.835555,0.818893,0.811074
2,0.302727,0.220103,0.116065,0.145368,0.039134,0.176603,1.0,1.0,1.0,0.948023,...,0.696746,1.134039,1.096800,1.103405,0.738494,0.746700,0.737424,0.818893,0.811074,0.816970
3,0.397344,0.069490,0.153757,0.042407,0.118485,0.218518,1.0,1.0,1.0,0.922293,...,0.694415,1.096800,1.103405,1.139278,0.746700,0.737424,0.742776,0.811074,0.816970,0.822994
4,0.147383,0.211486,0.061306,0.091608,0.296954,0.191264,1.0,1.0,1.0,0.915127,...,0.694900,1.103405,1.139278,1.176290,0.737424,0.742776,0.781663,0.816970,0.822994,0.824404


In [52]:

# Save to a new CSV file
data.to_csv('cleaned_state_action_recopilation.csv', index=False)


In [11]:
import pandas as pd
import numpy as np
import ast
import json

def clean_state_action_csv(input_file, output_file, config_file):
    # Load the config.json file
    with open(config_file, 'r') as f:
        config = json.load(f)
    
    # Extract values of M, L, and N from the config
    M = len(config['session']['codes']) + 1  # Number of assets + 1 for cash or risk-free asset
    L = int(config['session']['agents'][2])  # Look-back window length
    N = len(config['session']['features'])  # Number of features per asset
    
    # Load the CSV file
    df = pd.read_csv(input_file)
    
    # Function to parse the string representation of arrays
    def parse_array(array_str):
        return np.array(ast.literal_eval(array_str.replace('\n', '').replace('  ', ',').replace(' ', ',')))
    
    # Parse the State and Action columns
    df['State'] = df['State'].apply(parse_array)
    df['Action'] = df['Action'].apply(parse_array)
    
   # Flatten the State and Action columns
    flattened_data = []
    for _, row in df.iterrows():
        action = row['Action'].flatten()
        state = row['State'].flatten()
        flattened_row = np.concatenate((action, state))
        flattened_data.append(flattened_row)
    
    # Create column names
    action_columns = [f'Action{i+1}' for i in range(M)]
    state_columns = []
    for i in range(M):
        for j in range(L):
            for k in range(N):
                state_columns.append(f'State{i+1}.{j+1}.{k+1}')
    columns = action_columns + state_columns
    
    # Create a new DataFrame with the flattened data
    cleaned_df = pd.DataFrame(flattened_data, columns=columns)
    
    # Save the cleaned DataFrame to a new CSV file
    cleaned_df.to_csv(output_file, index=False)



In [12]:

# Example usage
input_file = 'state_action_recopilation.csv'
output_file = 'cleaned_state_action.csv'
config_file = 'config.json'

clean_state_action_csv(input_file, output_file, config_file)


In [15]:
import pandas as pd

def merge_state_action_results(cleaned_state_action_file, results_file, output_file,config_file):
    # Load the config.json file
    with open(config_file, 'r') as f:
        config = json.load(f)
    
    # Extract values of M, L, and N from the config
    M = len(config['session']['codes']) + 1  # Number of assets + 1 for cash or risk-free asset
    
    # Load the cleaned state-action CSV
    state_action_df = pd.read_csv(cleaned_state_action_file)
    
    # Load the results CSV
    results_df = pd.read_csv(results_file)
    
    # Drop the 'weight' column from the results CSV if it exists
    if 'Weight' in results_df.columns:
        results_df = results_df.drop(columns=['Weight'])
    
    price_columns = []
    # Divide the price column in M columns by the comma
    for i in range(M):
        price_columns.append(f'Price{i+1}')
    results_df[price_columns] = results_df['Price'].str.split(',', expand=True)
    results_df.drop('Price', axis=1, inplace=True)

    # Drop the first column of the results_df
    results_df.drop(results_df.columns[0], axis=1, inplace=True)

    # Merge the two dataframes on their indices (assuming they align correctly)
    merged_df = pd.concat([state_action_df, results_df], axis=1)
    
    # Save the merged dataframe to a new CSV file
    merged_df.to_csv(output_file, index=False)

# Example usage
cleaned_state_action_file = 'cleaned_state_action.csv'
results_file = 'result1-50.21499105705498.csv'
output_file = 'state_action_results.csv'
config_file = 'config.json'

merge_state_action_results(cleaned_state_action_file, results_file, output_file,config_file)
